In [1]:
from data_utils import load_rt

In [2]:
pos, neg = load_rt()

In [3]:
print(len(pos)); print(len(neg))

print(pos[0])

print(neg[0])

5331
5331
the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
simplistic , silly and tedious .


In [4]:
#find max length and pad all reviews below max length

max_len = 0

pos_and_neg = pos + neg

for pn in pos_and_neg:
    if len(pn.split()) > max_len:
        max_len = len(pn.split())
        
for i, _ in enumerate(pos):
    while len(pos[i].split()) < max_len:
        pos[i] += (" <PAD>")

for i, _ in enumerate(neg):
    while len(neg[i].split()) < max_len:
        neg[i] += (" <PAD>")
        
        
print(pos[0])
print(len(pos[0].split()))

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
59


In [5]:
#build vocab of unique words to turn into indices

pos_and_neg = pos + neg #re-define this as sequences are now padded!

vocab = set()

all_words = [w for pn in pos_and_neg for w in pn.split()]

vocab.update(all_words)

In [6]:
#build word to index dictionary

word2idx = { w:i for i,w in enumerate(vocab) }
idx2word = { i:w for i,w in enumerate(vocab) }

In [7]:
#convert words to indices and add labels, 1 for pos, 0 for neg

data = []

for p in pos:
    p_index = [word2idx[w] for w in p.split()]
    data.append([p_index, 1])
    
for n in neg:
    n_index = [word2idx[w] for w in n.split()]
    data.append([n_index, 0])
    
print(data[0])
print([idx2word[i] for i in data[0][0]])

[[20094, 12935, 8472, 15068, 15926, 10405, 20094, 10535, 14948, 13290, 16620, 9790, 16620, 15001, 2256, 1924, 20587, 15926, 6387, 13602, 20362, 9257, 7092, 17586, 16875, 8007, 20786, 3677, 16911, 17211, 19722, 20016, 12636, 7396, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213, 19213], 1]
['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', "century's", 'new', '"', 'conan', '"', 'and', 'that', "he's", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [8]:
#define network
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(Net, self).__init__()
    
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.fc1 = nn.Linear(59, 100)
        self.fc2 = nn.Linear(100, 2)
        
    def forward(self, x):
        #x = self.embedding(x)
        x = x.float()
        x = x.view(1, 59)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        return x

In [9]:
#define network

net = Net(len(vocab), 100)

In [10]:
#set hyperparameters 

epochs = 10
batch_size = 32
learning_rate = 0.001
momentum = 0.9

In [11]:
#define loss and optimizer

import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

In [12]:
from torch.autograd import Variable

print(net)

for e in range(1, epochs+1):
    
    for d in data:
        
        input_variable = Variable(torch.LongTensor(d[0]))

        if d[1] == 0:
            target_variable = Variable(torch.LongTensor([0]))
        else:
            target_variable = Variable(torch.LongTensor([1]))
            
        optimizer.zero_grad()

        fx = net(input_variable)

        loss = criterion(fx, target_variable)

        loss.backward()

        optimizer.step()
    
        print("Epoch %02d, loss = %f" % (e, loss.data[0]/(5331*2)))
    
    
    

Net (
  (embedding): Embedding(21402, 100)
  (fc1): Linear (59 -> 100)
  (fc2): Linear (100 -> 2)
)
Epoch 01, loss = -0.000094
Epoch 01, loss = 0.000000
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = 0.000000
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, l

Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = 0.000000
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Ep

Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
E

Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
E

Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
Epoch 01, loss = -0.000094
E

...

Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, lo

Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, lo

Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, lo

Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000
Epoch 01, loss = 0.000000


KeyboardInterrupt: 